In [2]:
import pandas as pd
import utils
import plotly.graph_objects as go

In [1]:
pair = "EUR_USD"
granularity = "H1"
ma_list = [8,16,32,64,128,256]

In [3]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [4]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [5]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [7]:
for ma in ma_list:  
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()

In [8]:
df_ma.dropna(inplace=True)

In [9]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_8,MA_16,MA_32,MA_64,MA_128,MA_256
255,2020-06-23T10:00:00.000000000Z,1.12882,1.12982,1.12835,1.12924,1.127903,1.127278,1.124938,1.122506,1.124152,1.127836
256,2020-06-23T11:00:00.000000000Z,1.12922,1.13073,1.12856,1.13060,1.128362,1.127593,1.125293,1.122668,1.124125,1.127840
257,2020-06-23T12:00:00.000000000Z,1.13062,1.13333,1.13062,1.13298,1.129215,1.128019,1.125687,1.122863,1.124123,1.127854
258,2020-06-23T13:00:00.000000000Z,1.13299,1.13382,1.13192,1.13290,1.130091,1.128428,1.126106,1.123057,1.124118,1.127868
259,2020-06-23T14:00:00.000000000Z,1.13289,1.13487,1.13266,1.13334,1.130883,1.128806,1.126540,1.123251,1.124107,1.127884


In [11]:
df_plot = df_ma.iloc[-200:].copy()

In [12]:
df_plot.shape

(200, 11)

In [13]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list:  
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
        ))
fig.update_layout(width=1000,height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()